<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/workshop_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('/content/workshop01-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.0/207.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.5/236.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.5/517.5 kB 28.8 MB/s eta 0:00:00
   ━

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips, get_segment_ids, get_radio_interfaces, get_worker_interfaces

# Reserve resources

In the next cell, we reserve 1 SDR and 1 worker.

In [ ]:
# Check the SDR's health and the status of its ports, both ports are supposed to be up, otherwise contact support
sdr_name = "sdr-02"
sdr_status = get_radio_interfaces(sdr_name)
logger.info(f"{json.dumps(sdr_status, indent=4)}")
for port in sdr_status.keys():
  if sdr_status[port]['linkstate'] == 'Down':
    logger.warning(f"port {port} on {sdr_name} is down.")
  if sdr_status[port]['linkstate'] == 'Up':
    logger.success(f"port {port} on {sdr_name} is up.")

2024-12-16 09:41:32.255 | INFO     | __main__:<cell line: 4>:4 - {
    "sdr_02_mango": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "On",
        "neg": "Enabled",
        "port": "te1/0/29",
        "segment_id": 103,
        "speed": "1000",
        "stitches": {},
        "type": "10G-Copper"
    },
    "sdr_02_ni": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "Off",
        "neg": "Disabled",
        "port": "te2/0/14",
        "segment_id": 104,
        "speed": "10000",
        "stitches": {},
        "type": "10G-Fiber"
    }
}
2024-12-16 09:41:32.257 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_02_mango on sdr-02 is up.
2024-12-16 09:41:32.258 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_02_ni on sdr-02 is up.


In [ ]:
# Reserve the SDR
segment_ids = get_segment_ids(sdr_name)

# reserve RJ45 port
rj45_lease = show_reservation_byname(sdr_name + "-rj45-lease")
if not rj45_lease:
    rj45_lease = reserve(
        { "type":"network", "name": sdr_name+"-rj45", "net_name": sdr_name+"-rj45", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# reserve SFP port
sfp_lease = show_reservation_byname(sdr_name + "-sfp-lease")
if not sfp_lease:
    sfp_lease = reserve(
        { "type":"network", "name": sdr_name+"-sfp", "net_name": sdr_name+"-sfp", "segment_id": segment_ids['sfp'], "duration": { "days":7, "hours":0 } }
    )

worker_name = 'worker-02'
# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# advantech router reservation
adv_name = "adv-02"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-12-16 09:43:21.331 | INFO     | chi.expeca:reserve:243 - reserving sdr-02-rj45
2024-12-16 09:43:23.661 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for sdr-02-rj45-lease with id d10eb695-35cd-4816-b89e-cc70116d29b6 to become "ACTIVE"
2024-12-16 09:43:28.811 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-02-rj45-lease with id d10eb695-35cd-4816-b89e-cc70116d29b6 is PENDING.
2024-12-16 09:43:33.980 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-02-rj45-lease with id d10eb695-35cd-4816-b89e-cc70116d29b6 is PENDING.
2024-12-16 09:43:39.149 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-02-rj45-lease with id d10eb695-35cd-4816-b89e-cc70116d29b6 is PENDING.
2024-12-16 09:43:44.320 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-02-rj45-lease with id d10eb695-35cd-4816-b89e-cc70116d29b6 is PENDING.
2024-12-16 09:43:49.487 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-02-rj45-

[
    {
        "name": "sdr-02-sfp-lease",
        "id": "391cf0c0-a5e3-4ad5-8cf0-54f7029ffd8d",
        "reservation_id": "8e3f72cf-55c6-471a-9b3a-ee36cf6cb0a9",
        "status": "ACTIVE",
        "end_date": "2024-12-23T09:44:00.000000"
    },
    {
        "name": "worker-02-lease",
        "id": "a8b7e69b-29ee-4503-a7af-7dd140b6875d",
        "reservation_id": "ecd0f887-c6a0-4a14-aa07-5608043544eb",
        "status": "ACTIVE",
        "end_date": "2024-12-23T09:45:00.000000"
    },
    {
        "name": "sdr-02-rj45-lease",
        "id": "d10eb695-35cd-4816-b89e-cc70116d29b6",
        "reservation_id": "245cd3a7-a214-4038-a960-56c9a89a7dc8",
        "status": "ACTIVE",
        "end_date": "2024-12-23T09:43:00.000000"
    }
]


# Run A Public Container to Test the SDR

Run a public container to test the SDR via its SFP port

In [ ]:
# check available 1Gbps interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    if interfaces[interface]['speed'] == '1000':
      available_ifs.append(interface)
logger.info(f"Available 1Gbps interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 1:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find enough 1Gbps interfaces on {worker_name}")

# run the container
advnet = chi.network.get_network(adv_name+"-net")
container_name = adv_name+"-end-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces on the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
# we need one 10Gbps interface for the SDR,
# and one port with any speed for the public interface
port_10g = None
port_any = None
for interface in available_ifs:
  if (port_10g is None) and (interfaces[interface]['speed'] == '10000'):
    port_10g = interface
    continue
  if port_any is None:
    port_any = interface
if (port_10g is None) or (port_any is None):
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find proper interfaces on {worker_name}")
else:
  logger.success(f"we choose {port_10g} for the SDR and {port_any} for the public access")

# run the container
sdrsfpnet = chi.network.get_network(sdr_name+"-sfp-net")
publicnet = chi.network.get_network("serverpublic")
container_name = "my-public-sdr-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-dind-sdr",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : sdrsfpnet['id'] },
    ],
    labels = {
        "networks.1.interface":port_any,
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
        "networks.2.interface":port_10g,
        "networks.2.ip":"10.30.10.120/24",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")


2024-12-16 10:03:18.206 | INFO     | __main__:<cell line: 6>:6 - Available public ips: ['130.237.11.116', '130.237.11.117', '130.237.11.122', '130.237.11.123', '130.237.11.125'].
2024-12-16 10:03:18.208 | INFO     | __main__:<cell line: 7>:7 - We choose 130.237.11.116 for this container.
2024-12-16 10:03:31.871 | INFO     | __main__:<cell line: 15>:15 - Available interfaces on worker-02: ['eno12409np1', 'ens5f0', 'ens5f1', 'ens6f0', 'ens6f1']
2024-12-16 10:03:31.873 | SUCCESS  | __main__:<cell line: 26>:30 - we choose eno12409np1 for the SDR and ens5f0 for the public access
2024-12-16 10:04:08.502 | SUCCESS  | __main__:<cell line: 60>:60 - created my-public-sdr-host container, reachable at 130.237.11.116.
